In [267]:
import pandas as pd
import numpy as np
import sklearn as sk

In [268]:
# importing data
pricing_requests = pd.read_csv("XYtrain.csv",index_col =0)
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
Xtest = pd.read_csv("test_set.csv",index_col =0)
Xtest = Xtest.join(hotels, on=['hotel_id', 'city'])

In [270]:
#adapting data for the AVN
Xdum = pricing_requests.drop(["price","avatar_id"],axis =1)
Xdum =pd.get_dummies(Xdum)

In [271]:
#sampling as many rows from request set as there is in the test set
Sample = Xdum.sample(n=6644)

In [272]:
#adding labels for classification
Sample["label"]=0
Xtest["label"]=1

In [273]:
#adapting data for the AVN
Xtest=Xtest.drop(["order_requests","avatar_id"],axis =1)
XtDum =pd.get_dummies(Xtest)
Data = pd.concat([Sample,XtDum])

In [275]:
#separating labels and the rest
y = Data["label"]
X = Data.loc[ : , Data.columns != 'label']

In [276]:
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense


X = pd.get_dummies(X)
y= np.array(y).reshape(-1, 1)

# Create the adversarial validation network
model = Sequential()
model.add(Dense(64, input_dim=X.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
416/416 [==============================] - 1s 904us/step - loss: 1.2004 - accuracy: 0.5935
Epoch 2/10
416/416 [==============================] - 0s 845us/step - loss: 0.6785 - accuracy: 0.6255
Epoch 3/10
416/416 [==============================] - 0s 881us/step - loss: 0.6400 - accuracy: 0.6604
Epoch 4/10
416/416 [==============================] - 0s 828us/step - loss: 0.6369 - accuracy: 0.6646
Epoch 5/10
416/416 [==============================] - 0s 689us/step - loss: 0.5952 - accuracy: 0.6938
Epoch 6/10
416/416 [==============================] - 0s 675us/step - loss: 0.5834 - accuracy: 0.6986
Epoch 7/10
416/416 [==============================] - 0s 703us/step - loss: 0.5934 - accuracy: 0.6930
Epoch 8/10
416/416 [==============================] - 0s 869us/step - loss: 0.5837 - accuracy: 0.7012
Epoch 9/10
416/416 [==============================] - 0s 748us/step - loss: 0.5646 - accuracy: 0.7134
Epoch 10/10
416/416 [==============================] - 0s 661us/step - loss: 0.554

In [278]:
#data that we will use for prediction
Xpredict = pricing_requests.drop(["price","avatar_id"],axis =1)
Xpredict = pd.get_dummies(Xpredict)

In [281]:
#prediction
pricing_requests["prediction"]= model.predict(Xpredict)

14716/14716 [==============================] - 7s 475us/step


In [357]:
#keeping all rows where the network thinks it belongs to the dataframe
df_classified_as_1 = pricing_requests[pricing_requests['prediction'] > 0.98]

In [358]:
df_classified_as_1

,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy,prediction
0,969,73,56,amsterdam,44,dutch,0,111612,Accar Hotels,Ibas,1,0,0,0.985930
23,909,58,56,madrid,44,dutch,0,111612,Yin Yang,8 Premium,1,0,0,0.986062
113,789,57,56,madrid,44,dutch,0,111612,Accar Hotels,Ibas,1,0,0,0.986327
115,915,57,56,madrid,44,dutch,0,111612,Accar Hotels,Ibas,1,0,0,0.990860
20,987,69,56,rome,44,dutch,0,111612,Accar Hotels,Ibas,1,0,0,0.984647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,101,415,0,amsterdam,0,romanian,0,303616,Accar Hotels,Safitel,1,0,0,0.980910
39,97,250,0,madrid,0,cypriot,1,303616,Accar Hotels,Safitel,1,1,0,0.991912
59,97,260,0,madrid,0,swedish,1,303616,Accar Hotels,Safitel,1,1,0,0.990749
98,888,231,10,madrid,0,swedish,1,303616,Morriott International,Corlton,1,0,1,0.982095


In [328]:
test = pd.read_csv("test_set.csv",index_col =0)

test = test.join(hotels, on=['hotel_id', 'city'])
test = test.drop(["order_requests"],axis =1)

In [330]:
#checking if all categories from test set are present
print(len(df_classified_as_1['city'].unique()))
print(len(df_classified_as_1["language"].unique()))
print(len(df_classified_as_1["group"].unique()))
print(df_classified_as_1["brand"].unique())

9
27
6
['Ibas' '8 Premium' 'Independant' 'Corlton' 'Tripletree' 'Navatel'
 'Morriot' 'Safitel' 'Chill Garden Inn' 'Boss Western' 'J.Halliday Inn'
 'Ardisson' 'Quadrupletree' 'Royal Lotus']


In [331]:
#checking if all categories are present
print(len(test['city'].unique()))
print(len(test["language"].unique()))
print(len(test["group"].unique()))
print(test["brand"].unique())

9
27
6
['J.Halliday Inn' 'Marcure' 'Independant' 'Ibas' 'Safitel' '8 Premium'
 'Tripletree' 'CourtYord' 'Royal Lotus' 'Boss Western' 'Corlton' 'Navatel'
 'Ardisson' 'Morriot' 'Chill Garden Inn' 'Quadrupletree']


In [359]:
#This cell can help you find rows with good probabilities for the missing categories
mask = pricing_requests['brand'].isin(['Marcure'])

selected_rows = pricing_requests[mask]
Add1 = selected_rows[max(selected_rows['prediction'])==selected_rows['prediction']]

In [360]:
mask = pricing_requests['brand'].isin(['CourtYord'])

selected_rows = pricing_requests[mask]
Add2 = selected_rows[max(selected_rows['prediction'])==selected_rows['prediction']]

In [361]:
#Adding it to the dataframe
df_classified_as_1 = pd.concat([df_classified_as_1,Add1,Add2])

In [362]:
#dropping all request that were done multiple times
df_classified_as_1 =df_classified_as_1.drop_duplicates()

In [363]:
df_classified_as_1

,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy,prediction
0,969,73,56,amsterdam,44,dutch,0,111612,Accar Hotels,Ibas,1,0,0,0.985930
23,909,58,56,madrid,44,dutch,0,111612,Yin Yang,8 Premium,1,0,0,0.986062
113,789,57,56,madrid,44,dutch,0,111612,Accar Hotels,Ibas,1,0,0,0.986327
115,915,57,56,madrid,44,dutch,0,111612,Accar Hotels,Ibas,1,0,0,0.990860
20,987,69,56,rome,44,dutch,0,111612,Accar Hotels,Ibas,1,0,0,0.984647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,888,243,14,madrid,2,swedish,1,303616,Morriott International,Corlton,1,0,1,0.986087
14,897,398,2,rome,2,estonian,1,303616,Chillton Worldwide,Chill Garden Inn,1,0,2,0.980518
2,888,220,12,madrid,1,slovakian,1,303616,Morriott International,Corlton,1,0,1,0.983751
131,29,361,4,paris,5,finnish,1,303616,Accar Hotels,Marcure,1,0,0,0.967985


In [364]:
df_classified_as_1= df_classified_as_1.drop(["prediction"],axis=1)

In [365]:
df_classified_as_1.to_csv("AVN_Data.csv")